## Collocate EarthCARE data with TCs

~23 min for 62 best-track storms

In [34]:
# General
import os
import glob

# Display
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm

# Data treatment
import xarray as xr
import numpy as np
# from scipy.interpolate import griddata
import pandas as pd 
from tqdm.notebook import tqdm
import ephem
import seaborn as sns
import re
from datetime import datetime
from pyproj import CRS, Geod

# Custom
import sys
sys.path.append('../ectools')
from ectools import ecio
from ectools import ecplot as ecplt
from ectools import colormaps

# Default parameters
fontsize = 16
mpl.rcParams.update({'font.size': fontsize})
mpl.rcParams['figure.figsize'] = (15, 10)
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = 'white'

In [35]:
### Paths & Params
PRMS = {
    'ibt_vars2keep':    ['usa_lon', 'usa_lat', 'dist2land', 'usa_wind', 'usa_r34', 'usa_r50', 'usa_r64', 'usa_rmw', 'storm_speed', 'storm_dir', 'name', 'sid'],
    'ibt_info2keep':    ['sid', 'basin', 'name', 'usa_atcf_id', 'usa_agency', 'iso_time', 'usa_status'],
    'coloc_radius':     5e5,
}

PTHS = {
    'ibt_pth': '/home/arthur/data/tropical_cyclones/ibtracs/30042025/IBTrACS.ALL.v04r01.nc',
    'ear_dir': '/home/arthur/data/earthCARE/full_datasets/AC__TC__2B/',
    'sav_dir': '/home/arthur/results/TCsLifeMonitFromObs/PRJCT_VERTICAL_WIND/01_COLLOCATE_EARTHCARE_TCs/AC__TC__2B/',
    # 'ear_dir': '/home/arthur/data/earthCARE/full_datasets/AC__TC__2B/*/*/*/*.h5'
    # 'sav_dir': '/home/arthur/results/TCsLifeMonitFromObs/PRJCT_VERTICAL_WIND/Test_EarthCARE_v02/',
}

In [36]:
# Open EarthCARE data
all_data = glob.glob(PTHS['ear_dir'] + '**/*.h5', recursive=True)
all_data.sort()
print(len(all_data))

25525


In [37]:
# IBTrACS
ds_ibt_all = xr.open_dataset(PTHS['ibt_pth'])

# Select variables
ds_ibt                        = ds_ibt_all[PRMS['ibt_vars2keep']]
ds_ibt[PRMS['ibt_info2keep']] = ds_ibt_all[PRMS['ibt_info2keep']].astype(str, copy=False) # Convert byte arrays to strings

# Select dates
start_date                    = np.datetime64('2024-07-30')
fsi                           = np.nanargmin(np.abs(ds_ibt['time'][:, 0] - start_date)) # First storm index
ds_ibt                        = ds_ibt.isel(storm=slice(fsi, None))

# Convert to m/s and m units
for speed in ['usa_wind', 'storm_speed']:
    ds_ibt[speed]  *= 0.5144
for radius in ['usa_r34', 'usa_r50', 'usa_r64', 'usa_rmw']:
    ds_ibt[radius] *= 1852
    
# Add mean radii
ds_ibt['mean_r34']       = ds_ibt.usa_r34.mean(dim='quadrant', skipna=True)
ds_ibt['mean_r50']       = ds_ibt.usa_r50.mean(dim='quadrant', skipna=True)
ds_ibt['mean_r64']       = ds_ibt.usa_r64.mean(dim='quadrant', skipna=True)

# Intensification and growing rates in IBTrACS
ds_ibt['vmx_smth']       = ds_ibt.usa_wind.rolling(date_time=4, center=True, min_periods=2).mean()
ds_ibt['r34_smth']       = ds_ibt.mean_r34.rolling(date_time=4, center=True, min_periods=2).mean()
ds_ibt['vmx_dot_smth']   = ds_ibt.vmx_smth.differentiate(coord='date_time') / 3 # m/s/hr
ds_ibt['r34_dot_smth']   = ds_ibt.r34_smth.differentiate(coord='date_time') / 3 # km/hr

# ==> Add LMI values
new                      = ds_ibt.fillna(-9999)
idx_lmi                  = new.vmx_smth.argmax(dim='date_time')
# LMI
ds_ibt['lmi']            = ds_ibt.usa_wind[:, idx_lmi]
ds_ibt['t_lmi']          = ds_ibt.time[:, idx_lmi]
# time since LMI
dt                       = (ds_ibt.time - ds_ibt.t_lmi)
dt_scds                  = dt.dt.days * 24 * 3600 + dt.dt.seconds # Nb of days + Nb of seconds (>= 0 and less than 1 day) for each element
ds_ibt['t_since_lmi']    = dt_scds # in seconds

# ==> Add 17.5 m/s values 
### BEFORE LMI
new2                     = ds_ibt.usa_wind.where(ds_ibt.t_since_lmi < 0).fillna(-9999)
# 17.5 m/s before LMI
idx_t17                  = abs(new2 - 17.5).argmin(dim='date_time')
ds_ibt['t_17']           = ds_ibt.time[:, idx_t17]
# time since 17.5 m/s
dt                       = (ds_ibt.time - ds_ibt.t_17)
dt_scds                  = dt.dt.days * 24 * 3600 + dt.dt.seconds # Nb of days + Nb of seconds (>= 0 and less than 1 day) for each element
ds_ibt['t_since_17']     = dt_scds # in seconds
### AFTER LMI
new3                     = ds_ibt.usa_wind.where(ds_ibt.t_since_lmi > 0).fillna(-9999)
# 17.5 m/s after LMI
idx_t17_end              = abs(new3 - 17.5).argmin(dim='date_time')
ds_ibt['t_17_end']       = ds_ibt.time[:, idx_t17_end]
# time since t_17_end
dt                       = (ds_ibt.time - ds_ibt.t_17_end)
dt_scds                  = dt.dt.days * 24 * 3600 + dt.dt.seconds # Nb of days + Nb of seconds (>= 0 and less than 1 day) for each element
ds_ibt['t_since_17_end'] = dt_scds # in seconds

# Future Vmax values
ds_ibt['vmx_6hr']        = ds_ibt.usa_wind.shift(shifts={'date_time': -2}) # vmx a 6  hrs
ds_ibt['vmx_12hr']       = ds_ibt.usa_wind.shift(shifts={'date_time': -4}) # vmx a 12 hrs
# # Proof that not all values are separated by 3 hours, but most of them are
# dt             = ds_ibt.time - ds_ibt.time.shift(shifts={'date_time': -1})
# dt_scds        = dt.dt.days * 24 * 3600 + dt.dt.seconds # Nb of days + Nb of seconds (>= 0 and less than 1 day) for each element
# values, counts = np.unique(dt_scds, return_counts=True)
# print(values, counts)

In [38]:
def path2datetime(path):
    '''
    Given a path like: /home/arthur/data/earthCARE/full_datasets/AC__TC__2B/2024/08/04/ECA_EXAA_AC__TC__2B_20240804T011644Z_20240805T094618Z_01044C.h5
    returns the datetime object corresponding to acquisition start time: datetime.datetime(2024, 08, 04, 01, 16)
    '''
    dt_as_list = re.split(r'(\d+)', os.path.basename(path))[3:6]
    dt_conc    = ''.join(dt_as_list)
    return datetime.strptime(dt_conc, "%Y%m%dT%H%M%S")

In [39]:
subset = [
    'usa_lon', 'usa_lat', 'dist2land',
    'usa_wind', 'usa_rmw', 'mean_r34', 'mean_r50', 'mean_r64',
    'vmx_smth', 'r34_smth', 'vmx_dot_smth', 'r34_dot_smth',
    'lmi', 't_since_lmi', 't_17', 't_since_17', 't_17_end', 't_since_17_end',
    'vmx_6hr', 'vmx_12hr',
    'storm_speed', 'storm_dir'
] 

In [40]:
## ITERATE ON BEST-TRACKS STORMS
for s in tqdm(ds_ibt.storm):
    ds_ibt_crt = ds_ibt.isel(storm=s)
    ibt_times  = ds_ibt_crt.time.where(ds_ibt_crt.usa_atcf_id != '').dropna(dim='date_time')
    for i, ibt_time in enumerate(ibt_times):

        ## OPEN A SUBSET OF EARTHCARE CANDIDATES
        # Get year, month, and day of current track point. So we can open relevant EarthCARE files, and only those, to speed up computations
        ibt_time_pd    = pd.to_datetime(ibt_time.data)
        year           = ibt_time_pd.year
        month          = ibt_time_pd.month
        day            = ibt_time_pd.day

        ## SELECT THEM BY TIME
        # Select current day, as well as the day before and the day after
        ear_data_bef   = PTHS['ear_dir'] + '*/' + f'{year}/{month:02d}/{day - 1:02d}/*.h5' # 1 day before
        ear_data_crt   = PTHS['ear_dir'] + '*/' + f'{year}/{month:02d}/{day:02d}/*.h5'     # Current day
        ear_data_aft   = PTHS['ear_dir'] + '*/' + f'{year}/{month:02d}/{day + 1:02d}/*.h5' # 1 day after
        # Concatenate all the candidates paths
        ear_candidates = glob.glob(ear_data_bef) + glob.glob(ear_data_crt) + glob.glob(ear_data_aft)
        ear_candidates.sort()
        # Compute the times of these candidates
        ear_times_crt  = [np.datetime64(path2datetime(pth)) for pth in ear_candidates]
        # Compute time diff. between these candidates and current track point, in minutes
        min_diffs      = np.array([float((ibt_time - ear_time) / np.timedelta64(1, 'm')) for ear_time in ear_times_crt]) 
        # Select only those candidates that are close in time to the current track point
        col_ear_files  = np.array(ear_candidates)[(min_diffs > -90) & (min_diffs <= 90)] # IBTrACS points are supposed to be given every 3 hours, so we look into that time window
        
        ### ITERATE OVER THESE CANDIDATES
        for k, candidate in enumerate(col_ear_files):
            dse               = xr.open_dataset(candidate, group='/ScienceData')

            # Interpolate track on EarthCARE average time
            dsi_interp        = ds_ibt_crt.assign_coords({'date_time': ds_ibt_crt['time']}).dropna(dim='date_time', subset=['usa_lat'])[subset].drop_indexes('date_time').interp(date_time=dse.time.mean())
        
            # Convert center coordinates to array
            lons_center       = np.ones_like(dse.longitude) * float(dsi_interp.usa_lon)
            lats_center       = np.ones_like(dse.latitude)  * float(dsi_interp.usa_lat)
            
            # Get the direction between target point and center point; 0° is North, 90° is East
            geod_wgs84        = CRS("epsg:4326").get_geod()
            forward_az, _, dr = geod_wgs84.inv(dse.longitude, dse.latitude, lons_center, lats_center)
            forward_az        = forward_az + 180 % 360 # Rotate so that 0° is North, 90° is East

            # Check that the closest point to the TC center is within the accepted collocaltion radius
            if np.min(dr) <= PRMS['coloc_radius']:
                # Add variables to the EarthCARE xr.Dataset
                dse['file']                         = candidate
                
                dse['distance_to_track_center_ibt'] = xr.DataArray(
                    data=dr,
                    dims=['along_track']
                )
                dse['closest_distance_to_track_center_ibt'] = np.min(dr)

                dse['id_ibt']                               = ds_ibt_crt.sid.item()
                dse['atcf_id_ibt']                          = ds_ibt_crt.usa_atcf_id[0].item()
                dse['lat_ibt']                              = float(dsi_interp.usa_lat)
                dse['lon_ibt']                              = float(dsi_interp.usa_lon)
                dse['dist2land_ibt']                        = float(dsi_interp.dist2land) * 1000 # meters
                
                dse['vmx_ibt']                              = float(dsi_interp.usa_wind)
                dse['rmx_ibt']                              = float(dsi_interp.usa_rmw)
                dse['r34_ibt']                              = float(dsi_interp.mean_r34)
                dse['r50_ibt']                              = float(dsi_interp.mean_r50)
                dse['r64_ibt']                              = float(dsi_interp.mean_r64)
                
                dse['vmx_smth_ibt']                         = float(dsi_interp.vmx_smth)
                dse['r34_smth_ibt']                         = float(dsi_interp.r34_smth)
                dse['vmx_dot_smth_ibt']                     = float(dsi_interp.vmx_dot_smth)
                dse['r34_dot_smth_ibt']                     = float(dsi_interp.r34_dot_smth)
                dse['lmi_ibt']                              = float(dsi_interp.lmi)
                dse['t_lmi_ibt']                            = ds_ibt_crt.t_lmi.data
                dse['t_since_lmi_ibt']                      = float(dsi_interp.t_since_lmi)
                dse['t_17_ibt']                             = ds_ibt_crt.t_17.data
                dse['t_since_17_ibt']                       = float(dsi_interp.t_since_17)
                dse['t_17_end_ibt']                         = ds_ibt_crt.t_17_end.data
                dse['t_since_17_end_ibt']                   = float(dsi_interp.t_since_17_end)
                dse['dt_17_ibt']                            = (ds_ibt_crt.t_17_end.data - ds_ibt_crt.t_17.data) / np.timedelta64(1, 's')
                dse['vmx_6hr_ibt']                          = float(dsi_interp.vmx_6hr)
                dse['vmx_12hr_ibt']                         = float(dsi_interp.vmx_12hr)
                dse['storm_speed_ibt']                      = float(dsi_interp.storm_speed)
                dse['storm_dir_ibt']                        = float(dsi_interp.storm_dir)

                # Save file
                dse.to_netcdf(PTHS['sav_dir'] + candidate.split('/')[-1][:-3] + '.nc')

  0%|          | 0/85 [00:00<?, ?it/s]

In [41]:
### Save in a listing file
list_of_files = os.listdir(PTHS['sav_dir'])
with open(PTHS['sav_dir'] + 'listing_collocated_files.txt', 'w') as f:
    for file in list_of_files:
        f.write(file + '\n')

In [16]:
ds_ibt.usa_wind[:, 50]

<xarray.DataArray 'usa_wind' (storm: 85)> Size: 340B
array([15.432   ,       nan, 20.576   ,       nan, 12.860001, 27.7776  ,
             nan, 36.008   , 32.9216  ,       nan, 46.296   ,       nan,
       38.58    , 45.7816  , 18.004   ,       nan, 54.012   ,       nan,
             nan, 36.008   ,       nan,       nan, 14.917601,       nan,
             nan,       nan,       nan,       nan,       nan, 23.148   ,
       30.864   ,       nan, 45.7816  , 51.440002, 38.58    ,       nan,
             nan,       nan, 34.979202, 23.148   ,       nan,       nan,
       18.004   ,       nan, 63.7856  ,       nan,       nan,       nan,
       50.925602, 22.1192  , 23.148   ,       nan, 27.7776  ,       nan,
       14.917601,       nan,       nan,       nan,       nan,       nan,
             nan, 50.925602,       nan,       nan, 20.0616  ,       nan,
       26.2344  , 63.7856  ,       nan,       nan,       nan, 54.012   ,
             nan,       nan,       nan,       nan, 23.148   , 18.004   ,
       27.7776  ,       nan, 36.008   ,       nan, 27.7776  ,       nan,
             nan], dtype=float32)
Coordinates:
    time     (storm) datetime64[ns] 680B 2024-08-06T12:00:00.000039936 ... NaT
    lat      (storm) float32 340B ...
    lon      (storm) float32 340B ...
Dimensions without coordinates: storm
Attributes:
    long_name:              Maximum sustained wind speed
    standard_name:          tropical_cyclone_maximum_sustained_wind_speed
    units:                  kts
    valid_min:              1
    valid_max:              250
    coverage_content_type:  physicalMeasurement

In [30]:
ds_ibt.usa_wind[:, 40].isel(storm=slice(-8, None))

<xarray.DataArray 'usa_wind' (storm: 8)> Size: 32B
array([18.004 , 23.148 ,     nan, 65.8432,     nan, 63.7856,     nan,
           nan], dtype=float32)
Coordinates:
    time     (storm) datetime64[ns] 64B 2025-03-13T06:00:00.000039936 ... NaT
    lat      (storm) float32 32B ...
    lon      (storm) float32 32B ...
Dimensions without coordinates: storm
Attributes:
    long_name:              Maximum sustained wind speed
    standard_name:          tropical_cyclone_maximum_sustained_wind_speed
    units:                  kts
    valid_min:              1
    valid_max:              250
    coverage_content_type:  physicalMeasurement

In [32]:
ds_ibt.isel(storm=slice(-8, None)).name

<xarray.DataArray 'name' (storm: 8)> Size: 4kB
array(['IVONE', 'JUDE', 'UNNAMED', 'COURTNEY', 'DIANNE', 'ERROL', 'TAM',
       'UNNAMED'], dtype='<U128')
Dimensions without coordinates: storm
Attributes:
    long_name:              Name of system
    description:            May be a combination of names from different agen...
    coverage_content_type:  thematicClassification

In [33]:
# Courtney and Errol